This jupyter notebook is used for rapid development.

In [ ]:
#!/usr/bin/env python3

# MIT License
#
# (C) Copyright [2022] Hewlett Packard Enterprise Development LP
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included
# in all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR
# OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE,
# ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
# OTHER DEALINGS IN THE SOFTWARE.

In [63]:
import os
import yaml
import regex
import json
import re


from urllib.parse import urlparse
import copy



In [64]:
OPEN_API_FILE = "example_data/swagger_v2.yaml"
TAVERN_FILE_DIR = "example_data/functional"
TAVERN_FILES = []
TAVERN_REGEX = "test_.*.tavern.yaml"
API_TARGET_URLS = ["{hsm_base_url}"]  #use this to specify variables you care about!
TAVERN_CONFIG = "example_data/tavern_global_config.yaml"


# yaml.add_multi_constructor(u'!bool', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)
# yaml.add_multi_constructor(u'!anything', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)
# yaml.add_multi_constructor(u'!anybool', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)

#if there are unknown tags, ignore them, return None; I am doing this is place of using safeloader which only uses known tags
class SafeLoaderIgnoreUnknown(yaml.SafeLoader):
    def ignore_unknown(self, node):
        return None


SafeLoaderIgnoreUnknown.add_constructor(None, SafeLoaderIgnoreUnknown.ignore_unknown)

# parse all the tavern test files

test_cases = []

for file in os.listdir(TAVERN_FILE_DIR):
    file_path = os.path.join(TAVERN_FILE_DIR, file)
    if os.path.isfile(file_path):
        if regex.search(TAVERN_REGEX, file) is not None:


            with open(file_path) as stream:
                #Im not using safe load, because I have a custom constructor to turn all unknown tags into Nones
                tavern_docs = yaml.load_all(stream, Loader=SafeLoaderIgnoreUnknown)
                for doc in tavern_docs:
                    test = doc["test_name"]

                    for stage in doc["stages"]:
                        test_data = {}
                        test_data["file"] = file
                        test_data["test_name"] = test
                        test_stage = test + " - " + stage["name"]
                        test_data["stage"] = stage["name"]
                        test_data["request_url"] = stage["request"]["url"]
                        test_data["request_method"] = stage["request"]["method"].upper()
                        test_cases.append(test_data)
                        #test_case = (test_stage, stage["request"]["url"], stage["request"]["method"])
                        #print(test_case)
                    # for k,v in doc.items():
                    #     print(k,v)

In [65]:
# parse the swagger doc
api_methods = []
with open(OPEN_API_FILE) as stream:
    try:
        swagger = yaml.safe_load(stream)
        for path in swagger["paths"]:
            for method in swagger["paths"][path]:
                endpoint = {}
                endpoint["method"] = method.upper()
                endpoint["url"] = path
                api_methods.append(endpoint)
    except yaml.YAMLError as exc:
        print(exc)
        1 / 0

# print(json.dumps(api_methods, indent=2))
# print(json.dumps(test_cases,indent=2))


In [66]:
#filter out any urls that dont have the expected URL string in them.
filtered_test_cases = []
for test_case in test_cases:
    for urls in API_TARGET_URLS:
        if urls in test_case["request_url"]:
            filtered_test_cases.append(test_case)


In [67]:
with open(TAVERN_CONFIG) as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        logging.error(exc)
        exit(1)

#todo not sure I need this file?

In [68]:
# request_url = "{hsm_base_url}/hsm/v2/State/Components/{xname}"
# vs  url "/State/Components/{xname}" -> There is no guarantee that the variable is the same.  I think I want to convert all {.*} into a magic string like: {VARIABLE}
#But need to do this AFTER the base url filtering...

for test_case in filtered_test_cases:
    test_case["request_url"] = regex.sub(r'\{(.*?)\}', "{VARIABLE}", test_case["request_url"])


for endpoint in api_methods:
    endpoint["url"] = regex.sub(r'\{(.*?)\}', "{VARIABLE}", endpoint["url"])



In [69]:
#TODO start to build an endpoints map

endpoints = {}

for ep in api_methods:
    if ep["url"] in endpoints:
        if ep["method"] not in endpoints[ep["url"]]:
            endpoints[ep["url"]].setdefault(ep["method"], [])

    else:
        method = {}
        method[ep["method"]] = []
        endpoints.setdefault(ep["url"], method)


# print(json.dumps(endpoints,indent=2))

In [73]:

new_endpoints = copy.deepcopy(endpoints)
for test_case in filtered_test_cases:
    parsed_url = urlparse(test_case["request_url"])
    method = test_case["request_method"]
    for endpoint, verb in endpoints.items():
        sub_endpoint = re.sub("{VARIABLE}", "[{}a-zA-Z0-9_]*",endpoint)
        sub_endpoint = sub_endpoint + "$"
        if re.search(sub_endpoint, parsed_url.path) is not None and method in verb:

            nd = endpoints[endpoint]
            nd[method].append(test_case)
            new_endpoints[endpoint] = nd



In [74]:

final_endpoints = {}
for endpoint, verb in new_endpoints.items():
    data = {}
    data["methods"] = verb
    data["counts"] = {}

    for method, test_case_list in verb.items():
        data["counts"][method] = len(test_case_list)
    final_endpoints[endpoint] = data


In [75]:
print(json.dumps(final_endpoints,indent=2))
# print(json.dumps(new_endpoints,indent=2))


{
  "/service/ready": {
    "methods": {
      "GET": []
    },
    "counts": {
      "GET": 0
    }
  },
  "/service/liveness": {
    "methods": {
      "GET": []
    },
    "counts": {
      "GET": 0
    }
  },
  "/service/values": {
    "methods": {
      "GET": []
    },
    "counts": {
      "GET": 0
    }
  },
  "/service/values/arch": {
    "methods": {
      "GET": []
    },
    "counts": {
      "GET": 0
    }
  },
  "/service/values/class": {
    "methods": {
      "GET": []
    },
    "counts": {
      "GET": 0
    }
  },
  "/service/values/flag": {
    "methods": {
      "GET": []
    },
    "counts": {
      "GET": 0
    }
  },
  "/service/values/nettype": {
    "methods": {
      "GET": []
    },
    "counts": {
      "GET": 0
    }
  },
  "/service/values/role": {
    "methods": {
      "GET": []
    },
    "counts": {
      "GET": 0
    }
  },
  "/service/values/subrole": {
    "methods": {
      "GET": []
    },
    "counts": {
      "GET": 0
    }
  },
  "/service/val

In [58]:
#create a CSV
import csv


report = []
for url, data in final_endpoints.items():
    for method, count in data["counts"].items():
        report.append((url, method, count))

with open('analysis_file2.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['url','method', 'count'])
    for row in report:
        csv_out.writerow(row)


[('/service/ready', 'GET', 0), ('/service/liveness', 'GET', 0), ('/service/values', 'GET', 0), ('/service/values/arch', 'GET', 0), ('/service/values/class', 'GET', 0), ('/service/values/flag', 'GET', 0), ('/service/values/nettype', 'GET', 0), ('/service/values/role', 'GET', 0), ('/service/values/subrole', 'GET', 0), ('/service/values/state', 'GET', 0), ('/service/values/type', 'GET', 0), ('/State/Components', 'GET', 78), ('/State/Components', 'POST', 0), ('/State/Components', 'DELETE', 0), ('/State/Components/{VARIABLE}', 'GET', 9), ('/State/Components/{VARIABLE}', 'PUT', 0), ('/State/Components/{VARIABLE}', 'DELETE', 0), ('/State/Components/ByNID/{VARIABLE}', 'GET', 3), ('/State/Components/BulkStateData', 'PATCH', 0), ('/State/Components/{VARIABLE}/StateData', 'PATCH', 0), ('/State/Components/BulkFlagOnly', 'PATCH', 0), ('/State/Components/{VARIABLE}/FlagOnly', 'PATCH', 0), ('/State/Components/BulkEnabled', 'PATCH', 0), ('/State/Components/{VARIABLE}/Enabled', 'PATCH', 0), ('/State/Co